In [8]:
import pandas as pd
from table_models import Site
from create_spatial_db import SpatialDB
from sqlalchemy.orm import sessionmaker
from sqlalchemy .dialects.postgresql import array
import json

In [9]:
data = pd.read_csv("~/Downloads/tri_20_mn.csv")
GRAMS_TO_POUNDS = 0.00220462

In [10]:
# columns have awkward names like -> 64. POTW - TOTAL TRANSFERS
# we want them to be nice like -> potw_total_transfers
for col in data.columns:
    split = col.split(".")  # separates out leading integer
    new = split[-1]  # captures only the last part of the split as column name
    new = new[1:].lower().replace(" ", "_").strip("_-_")  # some cleanup
    data.rename(columns={col: new}, inplace=True)  # rename

In [11]:
for col in data.columns:
    print(col)

year
trifd
frs_id
facility_name
street_address
city
county
st
zip
bia
tribe
latitude
longitude
horizontal_datum
parent_co_name
parent_co_db_num
standard_parent_co_name
federal_facility
industry_sector_code
industry_sector
primary_sic
sic_2
sic_3
sic_4
sic_5
sic_6
primary_naics
naics_2
naics_3
naics_4
naics_5
naics_6
doc_ctrl_num
chemical
elemental_metal_included
tri_chemical/compound_id
cas#
srs_id
clean_air_act_chemical
classification
metal
metal_category
carcinogen
pfas
form_type
unit_of_measure
fugitive_air
stack_air
water
underground
underground_cl_i
underground_c_ii-v
landfills
a_-_rcra_c_landfill
b_-_other_landfills
land_treatment
surface_impndmnt
a_-_rcra_surface_im
b_-_other_surface_i
other_disposal
on-site_release_total
potw_-_trns_rlse
potw_-_trns_trt
potw_-_total_transfers
m10
m41
m62
m40_metal
m61_metal
m71
m81
m82
m72
m63
m66
m67
m64
m65
m73
m79
m90
m94
m99
off-site_release_total
m20
m24
m26
m28
m93
off-site_recycled_total
m56
m92
off-site_energy_recovery_t
m40_non-metal
m

In [12]:
columns = ['trifd', 'facility_name', 'street_address', 'city', 'county', 'st', 'zip', 'latitude', 'longitude', 'industry_sector', 'chemical', 'carcinogen', 'unit_of_measure', 'total_releases', "fugitive_air", "stack_air", "water", "underground", "landfills", "surface_impndmnt", "land_treatment"]
data = data.filter(items=columns)

In [13]:
engine = SpatialDB.init()
Session = sessionmaker(engine)

In [14]:
sites = data['trifd'].unique().tolist()  # keys

for site in sites:
    df = data[data.trifd == site]  # subset data for site
    check_carcinogen = df.carcinogen.unique()  # array of values
    carcinogen = True if "YES" in check_carcinogen else False  # site flagged True if any chemical is carcinogenic
    chemicals = {}  # placeholder
    geom = f"POINT({df.longitude.unique()[0]} {df.latitude.unique()[0]})"  # PostGIS string
    
    # determine site release handling
    types = []
    if df.fugitive_air.sum() > 0 or df.stack_air.sum() > 0:
        types.append("AIR")
    
    if df.water.sum() > 0:
        types.append("WATER")
        
    if df.underground.sum() > 0 or df.landfills.sum() > 0 or df.surface_impndmnt.sum() > 0:
        types.append("LAND")
        
    total_releases = 0.  # value counter
    
    # get row for each chemical at the site, record its unit and total, add total to releases
    for chemical in df.chemical.unique():
        row = df[df.chemical == chemical]
        unit = row['unit_of_measure'].iloc[0]
        total = row['total_releases'].iloc[0]
        
        total_releases += total if unit == "Pounds" else total * GRAMS_TO_POUNDS
        
        chemicals[chemical] = {
            "unit": unit,
            "total": total
        }
    
    if len(types) == 0:
        types = [None]
        
    # instantiate row for this site
    this_site = Site(
        site_id=df.trifd.unique()[0],
        name=df.facility_name.unique()[0],
        address=df.street_address.unique()[0],
        city=df.city.unique()[0],
        county=df.county.unique()[0],
        state=df.st.unique()[0],
        zip=int(df.zip.unique()[0]),
        latitude=df.latitude.unique()[0],
        longitude=df.latitude.unique()[0],
        sector=df.industry_sector.unique()[0],
        carcinogen=carcinogen,
        chemicals=chemicals,
        release_types=array(types),
        total_releases=total_releases,
        geom=geom
    )
        
    # create row in db
    with Session() as s:
        with s.begin():
            s.add(this_site)
    

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "sites_pkey"
DETAIL:  Key (site_id)=(56726CNTRL25216) already exists.

[SQL: INSERT INTO sites (site_id, name, address, city, county, state, zip, latitude, longitude, sector, carcinogen, chemicals, release_types, total_releases, geom) VALUES (%(site_id)s, %(name)s, %(address)s, %(city)s, %(county)s, %(state)s, %(zip)s, %(latitude)s, %(longitude)s, %(sector)s, %(carcinogen)s, %(chemicals)s, ARRAY[NULL], %(total_releases)s, ST_GeomFromEWKT(%(geom)s))]
[parameters: {'site_id': '56726CNTRL25216', 'name': 'CENTRAL BOILER INC', 'address': '20502 160TH ST', 'city': 'GREENBUSH', 'county': 'ROSEAU', 'state': 'MN', 'zip': 56726, 'latitude': 48.63242, 'longitude': 48.63242, 'sector': 'Machinery', 'carcinogen': False, 'chemicals': '{"Diisocyanates": {"unit": "Pounds", "total": 0.0}}', 'total_releases': 0.0, 'geom': 'POINT(-96.160366 48.63242)'}]
(Background on this error at: https://sqlalche.me/e/14/gk